<a href="https://colab.research.google.com/github/Wata22109/Git-Playground/blob/main/%E8%AA%B2%E9%A1%8C05_26_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **プログラムの名称**
**帰宅効率化アプリ**
## **概要**

このプログラムは中之島方面に行く電車が現在時刻から何分後に寝屋川市に到着するか表示するプログラムです。また、到着するまでの時間によって学校から歩いても間に合うのか、急ぐべきなのかを表示します。間に合うかどうかの基準は学校から寝屋川市駅まで歩いて約14分として判断しています。急ぐ必要がない場合(電車の到着が現在時刻から10分未満で確実に間に合わない場合)、文字は赤色で表示されます。
下校用なので11時以降の時刻表しか設定していません。
平日限定のものとなっているため、土曜日、日曜日、祝日に使用しても対応しません。

## **操作方法**

実行するだけで大丈夫です。
時刻表を変更することもできます。

## **制作者のコメント**

学校から駅へ小走りで行って、到着ぎりぎりで電車が来る。ということが今まで何度もあったので、それを無くしたいと思いこのプログラムを作りました。
このプログラムは、平日の寝屋川市から中之島方面で萱島駅に止まる電車の時刻しか登録していないのでほぼ自分専用となっています。
timeモジュールを使って現在時刻や電車の到着時刻を表示するのに最も苦労しました。
また、バグが非常に多かったので修正するのにも多くの時間を費やしました。


## **作成時間**

作成時間：**約25時間**


In [ ]:
#@title 初期化 ( %reset -f )
%reset -f

In [ ]:
#@title サブ処理(各種ライブラリのインポート)

#@markdown **■ datetime モジュール：日付と時刻についての操作を行うことができる。**

#@markdown **■ pytz モジュール：日本のタイムゾーン情報を得ることができる。**

#@markdown **■ time モジュール：現在時刻を得たり、処理を一時停止させたりすることができる。**

#@markdown **■ IPython.display モジュール：IPython環境でのテキストなどの表示を行う。**

#@markdown ・display 関数：コードの実行結果をIPython環境の出力セルに表示することができる。

#@markdown ・HTMLクラス：HTMLタグを使用してテキストの装飾をすることができる。

#@markdown ・clear_output 関数：IPython環境の出力をリセットすることができる。

#プログラム
import datetime
import pytz
import time
from IPython.display import display, HTML, clear_output


In [ ]:
#@title サブ処理1(次の次の電車の時刻を求める関数の定義)

def get_next_next_train_time(now, timetable):
    japan_timezone = pytz.timezone('Asia/Tokyo')  #東京のタイムゾーン情報を取り入れる
    next_next_train = None #

    for time_str in timetable:
        train_time = datetime.datetime.strptime(time_str, '%H:%M')
        train_time = japan_timezone.localize(train_time).replace(year=now.year, month=now.month, day=now.day) #次の次に到着する電車の時刻と現在の年、月、日を合わせる

        if train_time.time() > now.time():
            if next_next_train is None:
               next_next_train = train_time
            elif train_time < next_next_train:
                 next_next_train = train_time

    return next_next_train

In [ ]:
#@title サブ処理2(メインの処理を行う関数の定義)

def get_time():
    japan_timezone = pytz.timezone('Asia/Tokyo') #東京のタイムゾーン情報を取り入れる
    now = datetime.datetime.now(japan_timezone)  #現在時刻(now)を日本時間にする

    time_text = (f'現在時刻:{now.hour}時{now.minute}分{now.second}秒') #時計

    # 中之島方面電車の時刻表(急行を除く)　(変更可能)
    timetable = [
        '11:10', '11:19', '11:28', '11:39', '11:48', '11:57', '12:10', '12:18', '12:27', '12:40', '12:48', '12:57',
        '13:10', '13:18', '13:27', '13:40', '13:49', '13:57', '14:10', '14:19', '14:27', '14:40', '14:49', '14:57',
        '15:10', '15:19', '15:29', '15:42', '15:54', '16:06', '16:18', '16:30', '16:42', '16:50', '16:54', '17:02',
        '17:06', '17:14', '17:18', '17:26', '17:30', '17:39', '17:42', '17:50', '17:54', '18:03', '18:10', '18:20',
        '18:27', '18:34', '18:40', '18:44', '18:53', '19:01', '19:09', '19:19', '19:30', '19:36', '19:41', '19:49',
        '19:52', '20:01', '20:05', '20:17', '20:26', '20:29', '20:41', '20:53', '21:05', '21:17', '21:29', '21:41',
        '21:54', '22:06', '22:21', '22:35', '22:51', '23:07', '23:22', '23:40', '23:57'
      ]
    next_train = None
    for time_str in timetable:
        train_time = datetime.datetime.strptime(time_str, '%H:%M')
        train_time = japan_timezone.localize(train_time).replace(year=now.year, month=now.month, day=now.day) #次に到着する電車の時刻と現在の年、月、日を合わせる

        #到着時刻が現在時刻より先であればそれをnext_trainとする
        if train_time.time() > now.time():
            next_train = train_time
            break

    next_next_train = None
    if next_train:
        #サブ処理1で定義した関数を使い次の次の電車の時刻を求める
        next_next_train = get_next_next_train_time(next_train, timetable)

    train_text = ""

    if next_train:
        #次のとその次の電車が何分後か求める
        time_difference = next_train - now
        minutes_until_next_train = int(time_difference.total_seconds() // 60)
        train_text = (f'次の電車まで 約{minutes_until_next_train+20} 分後、')
        if 10 > minutes_until_next_train+20:
            train_text = (f'<span style="color: red;">{train_text}</span>')
        if next_next_train:
            time_difference_next_next = next_next_train - now
            minutes_until_next_next_train = int(time_difference_next_next.total_seconds() // 60)
            train_text += (f'\n1本後の電車まで 約{minutes_until_next_next_train+20} 分後')
            if 10 > minutes_until_next_next_train+20:
                train_text = (f'<span style="color: red;">{train_text}</span>')
    #HTMLで表示
    display(HTML(f'<h1>{time_text}</h1>'))
    display(HTML(f'<h1>{train_text}</h1>'))

    #到着時刻によってどうすべきかを表示
    if next_train and minutes_until_next_train+20 >= 14:
        display(HTML('<h1>おそらく間に合うでしょう。</h1>'))
    elif next_train and 14>minutes_until_next_train+20 >= 10:
        display(HTML('<h1>急げば間に合うかもしれません。</h1>'))
    elif next_train and 10 > minutes_until_next_train+20:
        if next_next_train is not None:
            if minutes_until_next_next_train+20 >= 14:
                display(HTML('<h1>次の電車は間に合いません。1本後の電車であれば間に合うでしょう。</h1>'))
            elif 14 > minutes_until_next_next_train+20 >= 10:
                display(HTML('<h1>次の電車は間に合いません。1本後の電車になら乗れるかもしれません。</h1>'))
            else:
                display(HTML('<h1>次の電車も、もう一つ次の電車も間に合いません。</h1>'))
        else:
            display(HTML('<h1>次が最後の電車です。急いでください。</h1>'))
    else:
      display(HTML('<h1>今日の電車はもうありません。</h1>'))

In [ ]:
#@title メイン処理

# サブ処理2を1秒おきに繰り返す
while True:
    get_time()
    time.sleep(1)
    clear_output(wait=True)

KeyboardInterrupt: ignored